Advanced Experiments with striding
==============================

A few examples of manipulating strides in numpy arrays.

Numpy arrays are wrappers around a single contiguous block of data. The "strides" are what describes how the single block of data is interpreted as an n dimensional array.

For the most part, this is all an implimentation detail, happening under the hood at the C level. But numpy lets you manipulate the strides, and this allows some really powerful tricks.

In [ ]:
import numpy as np

In [ ]:
# A basic 2d array:
a = np.zeros((3,4))
a

In [ ]:
#flatten it:
a.flatten()

In [ ]:
# a basic 1d array
a

In [ ]:
# the strides:
a.strides

This means that you need to skip 8 bytes to get from one element to the next (a double is 8 bytes per value)

In [ ]:
a.itemsize

In [ ]:
# reshape the array
a.shape = (2,6)
a

In [ ]:
# now look at the strides
a.strides

Same bytes in the data block, but the strides define how it is to be interpreted.

You need to skip 48 bytes to get from the beginning of one row to the beginning of the next row (8 bytes per value, times 6 values per row)

And still 8 bytes from one element to the next in one row.

In [ ]:
# now a different data type
a = np.arange(12, dtype=np.uint8)

In [ ]:
a.strides

only one byte per element

In [ ]:
# reshape again
a.shape = (3,4)
a

In [ ]:
a.strides

So 4 bytes to get from one row to the next, and one from one element to the next.

In [ ]:
# make it 3-d
a.shape = (2,3,2)
a

In [ ]:
a.strides

so six bytes to get form one "slab" to the next, 2 to get from one row to next in that slab, and still 1 to get from item to item.

The fancy stuff
================

The ``stride_tricks`` module provides utilities to manipulate the strides of numpy arrays to do tricky things.

In [ ]:
a = np.arange(10, dtype=np.uint8)
a.shape = (2,5)
a

In [ ]:
a2 = np.lib.stride_tricks.as_strided(a, (8, 3), (1,1) )
a2

huh?

``as_strided(a, (8, 3), (1,1) )``

means: take the data in array, ``a``, and make a new array of shape (8,3) with strides (1,1).

So the array is 8 x 3 == 24 elements in size, but the underlying data has only 10 elements in it. This works because the strides are set to (1,1), so to find the first element of the nth row, you go n*1 bytes from the beginning:

 * the zeroth row starts at byte 0
 * the first row starts ar byte 1
 * etc..

Then to get the elements in each row you go one byte more:

 * the zeroth element in the zeroth row is at byte 0.
 * the first element in the zeroth row is at byte 1
 * the second element in the zeroth row is at byte 2
 
 * the zeroth element in the first row is at byte 1
 * the first element in the first row is at byte 2
 * the second element in the first row is at byte 3

So that's how we get a bigger array than the number of elements -- elements are re-used in multiple parts of the array.


In [ ]:
a2.strides

In [ ]:
a.shape, a.size

In [ ]:
a2.shape, a2.size

a and a2 share the same data, so if we change a, a2 changes also:

In [ ]:
a[0,2] = 6
a

In [ ]:
a2

Note how the 6 is reused in the first three rows...

Why do this weird stuff?
----------------------------

It lets you do tricks where you want to re-use the same values without actually needing to copy them. For instance, a moving average.

In [ ]:
a.mean(axis=0)

``filter_example.py`` has a couple simple filters, using stride tricks to impliment them.

In [ ]:
import filter_example

In [ ]:
# create some data
a = np.arange(25)
np.random.shuffle(a)
a

In [ ]:
filter_example.moving_average(a, 3)

In [ ]:
filter_example.scaled_by_max(a,4)